In [2]:
import pandas as pd
import json

In [3]:
with open('daten.json', 'r', encoding='utf-8') as datei:
    geladenes_dict = json.load(datei)

In [10]:

# Suche nach allen Charts, die nach dem 1.4.2024 erstellt wurden
from datetime import datetime

def finde_charts_nach_datum(daten, datum_grenze):
    """
    Findet alle Charts, die nach einem bestimmten Datum erstellt wurden.
    
    Args:
        daten: Die zu durchsuchende Datenstruktur
        datum_grenze: Das Grenzdatum als datetime-Objekt
        
    Returns:
        Eine Liste mit chart_ids
    """
    chart_ids = []
    
    def durchsuche_rekursiv(daten):
        if isinstance(daten, dict):
            # Prüfe, ob es sich um einen Chart handelt
            if "id" in daten and "createdAt" in daten:
                try:
                    # Konvertiere das Erstellungsdatum in ein datetime-Objekt
                    created_date = datetime.fromisoformat(daten["createdAt"].replace('Z', '+00:00'))
                    
                    # Prüfe, ob das Chart nach dem Grenzdatum erstellt wurde
                    if created_date > datum_grenze:
                        chart_ids.append(daten["id"])
                except Exception as e:
                    print(f"Fehler beim Parsen des Datums für Chart {daten.get('id')}: {e}")
            
            # Durchsuche alle Werte im Dict rekursiv
            for wert in daten.values():
                durchsuche_rekursiv(wert)
                
        elif isinstance(daten, list):
            # Durchsuche alle Elemente in der Liste rekursiv
            for element in daten:
                durchsuche_rekursiv(element)
    
    durchsuche_rekursiv(daten)
    return chart_ids

# Setze das Grenzdatum auf den 1.4.2024
grenzdatum = datetime(2024, 4, 1, tzinfo=datetime.now().astimezone().tzinfo)

# Suche nach Charts, die nach dem 1.4.2024 erstellt wurden
neue_charts = finde_charts_nach_datum(geladenes_dict, grenzdatum)

# Ausgabe der Ergebnisse
print(f"Gefundene Charts nach dem 1.4.2024: {len(neue_charts)}")
print("\nListe der Chart-IDs:")
for chart_id in neue_charts:
    print(chart_id)

Gefundene Charts nach dem 1.4.2024: 2933

Liste der Chart-IDs:
9Uo9K
01moT
089Vk
0CWQ0
0FacC
0Ga38
0Hbdi
0Ip5v
0JUUR
0KqmN
0N6O8
0USsM
0Wued
0WuvR
0XF5f
0XKrt
0Z1Y9
0Z2Nf
0ZC8D
0aVnK
0c3IU
0gisZ
0irDM
0k9GQ
0lFsw
0mVhP
0qLGS
0sbd5
0wTcy
0yPMa
0ydUL
12M5D
12pjW
14QR9
17beO
1Axxu
1DPtL
1JcYi
1QDm9
1QsGg
1SE2L
1T095
1U0b4
1W0IV
1Wiyo
1XjEC
1Yosv
1aJtF
1as0Y
1kwXi
1nbtq
1pjA6
1tuIW
1uchl
1v57S
1vq7o
1wCKN
1xtPx
250Jw
265WK
2CnMA
2DiUy
2E96e
2NQty
2SRUi
2ZBKt
2dL1T
2gVz1
2kXM7
2uTL5
2vy5X
2xPA7
2ykuR
30kCy
32aJL
36cAb
3A2dE
3E0se
3Ey39
3Fieh
3FuBB
3Fze1
3H4UK
3KCYN
3LrCJ
3PGi1
3Q0XN
3RDdS
3TfoX
3UpgH
3VGZX
3W3Z6
3XBSi
3XHnX
3XlB7
3b7LE
3eXkw
3gUkJ
3iV4s
3ir79
3kTT5
3tUwN
3vpGA
3vts3
3xzyp
46JBK
4A315
4ApQx
4Ble5
4EO9b
4FfFG
4GeC6
4I1OY
4LSM5
4MaJW
4MsI6
4PRV7
4Q0Xr
4Qpon
4WWCT
4bsie
4c3mH
4iQcl
4uIbK
4xCqs
531Qu
559Kc
5Dg3i
5HFbm
5I8Ms
5Ih1u
5IpnV
5Ky8Y
5NNSe
5QIok
5RB6L
5UvZW
5VwMw
5XxRK
5adqf
5cL2f
5ekgD
5h8Kx
5hMnT
5mJxe
5nyYT
5oFJi
5uQ37
5wUre
5ywrN
60CSB
60wTg
61J9F
61iiE
61wiY
62FwM
6

In [12]:
len(neue_charts)

2933